In [2]:
%matplotlib inline

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 16:30:19.892843. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [1]:
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 16:30:16.814091. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [3]:
def gini(actual, pred):
    assert (len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    totalLosses = all[:, 0].sum()
    giniSum = all[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)


def gini_normalized(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 16:30:21.526065. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## One-Hot Encoding 

#### Load data 

In [4]:
data = pd.read_csv("../data/train.csv", na_values=[-1, -1.0], index_col="id")

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 16:30:31.181748. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [5]:
X, y = data.loc[:, data.columns != "target"], data.target

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 16:30:34.698861. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


#### Preprocess 

In [15]:
for col in X.columns[X.columns.str.endswith("cat")]:
    X.loc[:, col] = X.loc[:, col].astype("category")

/usr/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 16:35:21.176861. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [19]:
X = pd.get_dummies(X)

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 16:36:28.845012. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 16:36:38.821604. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


#### Balance 

In [22]:
%%time
resampler = RandomUnderSampler()
indexes = X_train.index.values.reshape(-1, 1)
index_res, y_res = resampler.fit_sample(indexes, y_train)

CPU times: user 154 ms, sys: 6.97 ms, total: 161 ms
Wall time: 159 ms


/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 16:36:41.793901. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [24]:
X_res = X_train.loc[index_res.flatten(), :]

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-11-12 16:40:42.473162. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


#### Train 

In [27]:
model = XGBClassifier()
model.fit(X_res, y_res)
y_pred = model.predict(X_test)

mean_gini = gini_normalized(y_test, y_pred)
print(mean_gini)

0.192635755713
